# 3. Préparation pour la Modélisation

Ce notebook se concentre sur la troisième étape du processus d'analyse prédictive : la préparation des données pour la modélisation. Nous allons séparer les données en ensembles d'entraînement et de test, standardiser les variables numériques, et sauvegarder les données préparées pour les utiliser dans les notebooks de modélisation.

## 3.1 Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# Bibliothèques pour la préparation des données
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Pour une meilleure lisibilité des graphiques
%matplotlib inline

# Configuration de l'aléatoire pour la reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 3.2 Chargement des données préparées

In [ ]:
# Chemin vers les données préparées
data_dir = "/home/ubuntu/notebooks/data"
data_path = os.path.join(data_dir, "credit_data_prepared.pkl")

# Vérification de l'existence du fichier
if os.path.exists(data_path):
    print(f"Le fichier existe à l'emplacement : {data_path}")
    # Chargement des données
    df = pd.read_pickle(data_path)
    print(f"Données chargées avec succès. Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
else:
    print(f"Erreur : Le fichier n'existe pas à l'emplacement : {data_path}")
    # Si le fichier n'existe pas, on charge les données brutes et on applique les transformations
    print("Tentative de chargement des données brutes...")
    excel_path = "/home/ubuntu/upload/ab4e5df7-08a5-4e76-8ddd-9d4f845ecff1.xlsx"
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        # Renommage des colonnes
        colonnes_renommees = {
            "client's age": "age",
            "marital status": "marital_status",
            "amount of expenses": "expenses",
            "amount of income": "income",
            "amount requested of loan": "loan_amount",
            "price of good": "good_price",
            "credit status": "credit_status"
        }
        df = df.rename(columns=colonnes_renommees)
        # Transformation de la variable cible
        df['credit_status'] = df['credit_status'].map({"Yes": 0, "No": 1})
        print(f"Données brutes chargées et transformées. Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
    else:
        print(f"Erreur : Le fichier de données brutes n'existe pas non plus à l'emplacement : {excel_path}")

## 3.3 Aperçu des données

In [ ]:
# Affichage des premières lignes
print("Aperçu des 5 premières lignes :")
df.head()

In [ ]:
# Informations sur les colonnes
print("Informations sur les colonnes :")
df.info()

In [ ]:
# Vérification des valeurs manquantes
print("Vérification des valeurs manquantes :")
df.isnull().sum()

## 3.4 Ingénierie des caractéristiques

Basé sur l'analyse exploratoire, nous allons créer quelques caractéristiques supplémentaires qui pourraient être utiles pour la modélisation.

In [ ]:
# Création du ratio prêt/revenu
df['loan_to_income'] = df['loan_amount'] / df['income']

# Création du ratio prix du bien/prêt
df['good_price_to_loan'] = df['good_price'] / df['loan_amount']

# Création du ratio dépenses/revenu
df['expenses_to_income'] = df['expenses'] / df['income']

# Affichage des nouvelles caractéristiques
print("Aperçu des données avec les nouvelles caractéristiques :")
df.head()

In [ ]:
# Statistiques descriptives des nouvelles caractéristiques
print("Statistiques descriptives des nouvelles caractéristiques :")
df[['loan_to_income', 'good_price_to_loan', 'expenses_to_income']].describe()

In [ ]:
# Vérification des valeurs infinies ou NaN dans les nouvelles caractéristiques
print("Vérification des valeurs infinies ou NaN dans les nouvelles caractéristiques :")
print(f"Nombre de valeurs infinies dans loan_to_income : {np.isinf(df['loan_to_income']).sum()}")
print(f"Nombre de valeurs NaN dans loan_to_income : {np.isnan(df['loan_to_income']).sum()}")
print(f"Nombre de valeurs infinies dans good_price_to_loan : {np.isinf(df['good_price_to_loan']).sum()}")
print(f"Nombre de valeurs NaN dans good_price_to_loan : {np.isnan(df['good_price_to_loan']).sum()}")
print(f"Nombre de valeurs infinies dans expenses_to_income : {np.isinf(df['expenses_to_income']).sum()}")
print(f"Nombre de valeurs NaN dans expenses_to_income : {np.isnan(df['expenses_to_income']).sum()}")

In [ ]:
# Traitement des valeurs infinies ou NaN si nécessaire
# Remplacement des valeurs infinies par la valeur maximale non infinie
for col in ['loan_to_income', 'good_price_to_loan', 'expenses_to_income']:
    if np.isinf(df[col]).sum() > 0 or np.isnan(df[col]).sum() > 0:
        # Calcul de la valeur maximale non infinie et non NaN
        max_val = df[col][~np.isinf(df[col]) & ~np.isnan(df[col])].max()
        # Remplacement des valeurs infinies par la valeur maximale
        df[col] = df[col].replace([np.inf, -np.inf], max_val)
        # Remplacement des valeurs NaN par la médiane
        df[col] = df[col].fillna(df[col].median())
        print(f"Valeurs infinies et NaN remplacées dans {col}")

# Vérification après traitement
print("\nVérification après traitement :")
print(f"Nombre de valeurs infinies dans loan_to_income : {np.isinf(df['loan_to_income']).sum()}")
print(f"Nombre de valeurs NaN dans loan_to_income : {np.isnan(df['loan_to_income']).sum()}")
print(f"Nombre de valeurs infinies dans good_price_to_loan : {np.isinf(df['good_price_to_loan']).sum()}")
print(f"Nombre de valeurs NaN dans good_price_to_loan : {np.isnan(df['good_price_to_loan']).sum()}")
print(f"Nombre de valeurs infinies dans expenses_to_income : {np.isinf(df['expenses_to_income']).sum()}")
print(f"Nombre de valeurs NaN dans expenses_to_income : {np.isnan(df['expenses_to_income']).sum()}")

## 3.5 Séparation des données

Nous allons maintenant séparer les données en variables explicatives (X) et variable cible (y), puis en ensembles d'entraînement et de test.

In [ ]:
# Définition des variables explicatives et de la variable cible
# Nous incluons toutes les variables sauf credit_status
X = df.drop('credit_status', axis=1)
y = df['credit_status']

# Affichage des dimensions
print(f"Dimensions de X : {X.shape}")
print(f"Dimensions de y : {y.shape}")

# Affichage des noms des variables explicatives
print("\nVariables explicatives :")
print(X.columns.tolist())

In [ ]:
# Séparation en ensembles d'entraînement et de test (80% entraînement, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Affichage des dimensions
print(f"Dimensions de X_train : {X_train.shape}")
print(f"Dimensions de X_test : {X_test.shape}")
print(f"Dimensions de y_train : {y_train.shape}")
print(f"Dimensions de y_test : {y_test.shape}")

In [ ]:
# Vérification de la distribution de la variable cible dans les ensembles d'entraînement et de test
print("Distribution de la variable cible dans l'ensemble d'entraînement :")
print(y_train.value_counts(normalize=True) * 100)

print("\nDistribution de la variable cible dans l'ensemble de test :")
print(y_test.value_counts(normalize=True) * 100)

## 3.6 Standardisation des variables numériques

Nous allons standardiser les variables numériques pour qu'elles aient une moyenne de 0 et un écart-type de 1. Cela est important pour de nombreux algorithmes d'apprentissage automatique, en particulier ceux qui sont basés sur la distance comme KNN.

In [ ]:
# Création d'un scaler
scaler = StandardScaler()

# Ajustement du scaler sur l'ensemble d'entraînement uniquement
scaler.fit(X_train)

# Transformation des ensembles d'entraînement et de test
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Affichage des dimensions
print(f"Dimensions de X_train_scaled : {X_train_scaled.shape}")
print(f"Dimensions de X_test_scaled : {X_test_scaled.shape}")

In [ ]:
# Conversion des arrays numpy en DataFrames pour une meilleure lisibilité
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# Affichage des premières lignes des données standardisées
print("Aperçu des données standardisées (ensemble d'entraînement) :")
X_train_scaled_df.head()

In [ ]:
# Vérification des statistiques des données standardisées
print("Statistiques des données standardisées (ensemble d'entraînement) :")
X_train_scaled_df.describe().T[['mean', 'std', 'min', 'max']]

## 3.7 Sauvegarde des données préparées pour la modélisation

Nous allons sauvegarder les ensembles d'entraînement et de test, ainsi que le scaler, pour les utiliser dans les notebooks de modélisation.

In [ ]:
# Création d'un dossier pour les données de modélisation
model_data_dir = "/home/ubuntu/notebooks/model_data"
os.makedirs(model_data_dir, exist_ok=True)

# Sauvegarde des ensembles d'entraînement et de test
joblib.dump(X_train, os.path.join(model_data_dir, "X_train.pkl"))
joblib.dump(X_test, os.path.join(model_data_dir, "X_test.pkl"))
joblib.dump(y_train, os.path.join(model_data_dir, "y_train.pkl"))
joblib.dump(y_test, os.path.join(model_data_dir, "y_test.pkl"))

# Sauvegarde des ensembles standardisés
joblib.dump(X_train_scaled, os.path.join(model_data_dir, "X_train_scaled.pkl"))
joblib.dump(X_test_scaled, os.path.join(model_data_dir, "X_test_scaled.pkl"))

# Sauvegarde du scaler
joblib.dump(scaler, os.path.join(model_data_dir, "scaler.pkl"))

# Sauvegarde des noms des variables explicatives
with open(os.path.join(model_data_dir, "feature_names.txt"), "w") as f:
    f.write("\n".join(X.columns.tolist()))

print(f"Données préparées pour la modélisation sauvegardées avec succès dans le dossier : {model_data_dir}")

## 3.8 Visualisation de la distribution des données standardisées

In [ ]:
# Visualisation de la distribution des variables standardisées
plt.figure(figsize=(15, 10))

# Sélection de quelques variables importantes pour la visualisation
selected_features = ['age', 'income', 'loan_amount', 'good_price', 'loan_to_income']
selected_indices = [X_train.columns.get_loc(feature) for feature in selected_features]

for i, idx in enumerate(selected_indices):
    plt.subplot(2, 3, i+1)
    plt.hist(X_train_scaled[:, idx], bins=30, alpha=0.7)
    plt.title(f'Distribution de {X_train.columns[idx]} standardisée')
    plt.axvline(x=0, color='r', linestyle='--')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3.9 Analyse des corrélations après ingénierie des caractéristiques

In [ ]:
# Création d'un DataFrame avec les variables explicatives et la variable cible
df_corr = pd.DataFrame(X_train, columns=X_train.columns)
df_corr['credit_status'] = y_train.values

# Calcul de la matrice de corrélation
correlation_matrix = df_corr.corr()

# Visualisation de la matrice de corrélation
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice de corrélation après ingénierie des caractéristiques', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Corrélations avec la variable cible
target_correlations = correlation_matrix['credit_status'].sort_values(ascending=False)
print("Corrélations avec la variable cible (credit_status) :")
target_correlations

## 3.10 Résumé de la préparation pour la modélisation

Dans ce notebook, nous avons effectué les opérations suivantes :

1. **Chargement des données préparées** : Nous avons chargé les données préparées dans le notebook précédent.

2. **Ingénierie des caractéristiques** : Nous avons créé de nouvelles caractéristiques qui pourraient être utiles pour la modélisation :
   - `loan_to_income` : Ratio entre le montant du prêt et le revenu
   - `good_price_to_loan` : Ratio entre le prix du bien et le montant du prêt
   - `expenses_to_income` : Ratio entre les dépenses et le revenu

3. **Séparation des données** : Nous avons séparé les données en variables explicatives (X) et variable cible (y), puis en ensembles d'entraînement (80%) et de test (20%).

4. **Standardisation des variables numériques** : Nous avons standardisé les variables numériques pour qu'elles aient une moyenne de 0 et un écart-type de 1.

5. **Sauvegarde des données préparées** : Nous avons sauvegardé les ensembles d'entraînement et de test, ainsi que le scaler, pour les utiliser dans les notebooks de modélisation.

6. **Analyse des corrélations** : Nous avons analysé les corrélations entre les variables après l'ingénierie des caractéristiques pour identifier les variables les plus importantes pour la modélisation.

### Observations importantes :

1. **Variables les plus corrélées avec la variable cible** :
   - `loan_to_income` : Cette nouvelle caractéristique est fortement corrélée avec le statut de crédit, ce qui confirme l'importance du ratio entre le montant du prêt et le revenu.
   - `loan_amount` : Le montant du prêt est positivement corrélé avec le statut de non-solvabilité.
   - `income` : Le revenu est négativement corrélé avec le statut de non-solvabilité.

2. **Distribution équilibrée** : Nous avons utilisé la stratification lors de la séparation des données pour maintenir la même distribution de la variable cible dans les ensembles d'entraînement et de test.

3. **Standardisation réussie** : Les variables standardisées ont bien une moyenne proche de 0 et un écart-type proche de 1, ce qui est important pour les algorithmes sensibles à l'échelle des variables.

## Prochaine étape

Dans les prochains notebooks, nous allons développer et évaluer deux modèles de classification :
1. Régression logistique
2. K-Nearest Neighbors (KNN)

Nous comparerons ensuite leurs performances pour déterminer le meilleur modèle pour prédire le statut de crédit des clients.